In [ ]:
import sys, os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.regularizers import l2
import matplotlib.pyplot as plt

#loading previously saved features and labels
a = np.load('./features.npy')
B = np.load('./labels.npy')
#taking the mean of the pixel features
a -= np.mean(A, axis = 0)
#taking the normal standard deviation of the mean of the pixel features
a /= np.std(A, axis = 0)

#Sklearn's test train split function using pseudo random numbers so that everytime the code is run, the output of the split is the same. 80/20 split train/test
A_train, A_test, B_train, B_test = train_test_split(a, B, test_size = 0.1, random_state = 5)
#making a seperate split within the train split, to give a validation set
A_train, A_valid, B_train, B_valid = train_test_split(A_train, B_train, test_size = 0.1, random_state = 5)

#saving test split using a numpy array to be used later
np.save('ftestFear', A_test)
np.save('ltestFear', B_test)

#cnn model design
model = Sequential()

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', input_shape = (48, 48, 1), data_format = 'channels_last', kernel_regularizer = l2(0.01)))
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation = 'softmax'))

model.summary()

#Compliling the model with adam optimizer and categorical crossentropy loss
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

model_info = model.fit(np.array(A_train), np.array(B_train),
          batch_size = 64,
          epochs = 100,
          verbose = 1,
          validation_data = (np.array(A_valid), np.array(B_valid)),
          callbacks = [EarlyStopping(monitor = 'val_loss', patience = int(5))],
          shuffle = True)
#saving the  model to be used later for testing
CNN_json = model.to_json()
with open("CNNFear.json", "w") as json_file:
    json_file.write(CNN_json)
    model.save_weights("CNNFear.h5")
    
 #plotting results of the training
plt.plot(model_info.history['loss'])
plt.plot(model_info.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc = 'upper left') 
plt.show()